In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime


%matplotlib inline  

In [2]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'

In [3]:
''' Колонки '''
# ticker = 'BABA'

tickers_list = ['ADBE', 'COP', 'AMAT', 'QRVO', 'SPG', 'DRI', 'BA', 'META', 'NVDA', 'FDX', 'ADSK', 'ABMD', 'GS']
# ['ADBE', 'BAC', 'LVS', 'UAL', 'HOG']
required_year = '2022'

''' время '''
stime='1999-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [4]:
def one_stock_analyse(ticker, api_key, required_year):

    def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='1999-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    df = get_data_from_ticker(ticker)



    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)


    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr

    
    
    df_key_metr = get_key_metricks(ticker, api_key, required_year)
    
    
    if df_key_metr.index[0] == required_year:
        last_year = True
    else:
        last_year = False



    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)



    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            x += 1
            x = str(x)
            r = df.loc[f'{x}-01-01']
            r = float(r)
            new_list.append(r)

        return new_list

    stock_year_prices = get_price_from_df(df, index_years)


    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)

    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df
    
    df_with_changes = change_percent_all_columns(df_key_metr)
    
    if last_year == True:
        # df_last = df_with_changes.reindex(index=df_with_changes.index[::-1])
        r = [ticker, df_with_changes[f'change % {ticker}'][0], df_with_changes['change % debtToAssets'][0]]
    else:
        r = None 
        
        
    
    return print(r)

results

In [5]:
def get_results_da_price(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        r = one_stock_analyse(tickers_list[i], api_key, required_year)
        if r == None:
            pass
        else:
            l.append(r)
    return l
  
get_results_da_price(tickers_list, api_key, required_year)

['ADBE', -34.600411630333255, 5.678955876569617]
None
['AMAT', -19.67016020385691, 3.4179562492896594]
['QRVO', -25.80856688535968, 9.667965013725052]
None
['DRI', 10.98550362184325, 6.401377290781099]
None
None
['NVDA', -30.920513295999612, -3.75379776919074]
['FDX', -21.964417156613848, 0.27633585813470063]
['ADSK', -19.023976256421946, 3.916738051544556]


KeyError: '2023-01-01'